In [ ]:
import pandas as pd
import numpy as np
import uproot
import ROOT

def analysis(filename, treename):
    with uproot.open(filename) as file:
        df = file[treename].arrays(['pip1_E', 'pip1_px', 'pip1_py', 'pip1_pz', 'pip2_E', 'pip2_px', 'pip2_py', 'pip2_pz', 'pim_E', 'pim_px', 'pim_py', 'pim_pz', 'km_E', 'km_px', 'km_py', 'km_pz', 'p_E', 'p_px', 'p_py', 'p_pz'], library="pd")
        
        ######## DEFINITIONS ########

        # pi_plus (pip2) i.e. one of two components of kShort.  the other is pi_minus (pim)
        # total mass
        df['pip2_m'] = np.sqrt(df['pip2_E']**2 - df['pip2_px']**2 - df['pip2_py']**2 - df['pip2_pz']**2)
        # mass squared
        df['pip2_m2'] = df['pip2_E']**2 - df['pip2_px']**2 - df['pip2_py']**2 - df['pip2_pz']**2

        # pi_minus (pim) i.e. one of two components of kShort.  the other is pi_plus (pip2)
        # total mass
        df['pim_m'] = np.sqrt(df['pim_E']**2 - df['pim_px']**2 - df['pim_py']**2 - df['pim_pz']**2)
        # mass squared
        df['pim_m2'] = df['pim_E']**2 - df['pim_px']**2 - df['pim_py']**2 - df['pim_pz']**2

        # kShort (pip2 + pim)
        # 4-vector components
        df['ks_E'] = df['pip2_E'] + df['pim_E']
        df['ks_px'] = df['pip2_px'] + df['pim_px']
        df['ks_py'] = df['pip2_py'] + df['pim_py']
        df['ks_pz'] = df['pip2_pz'] + df['pim_pz']
        # total mass
        df['ks_m'] = np.sqrt(df['ks_E']**2 - df['ks_px']**2 - df['ks_py']**2 - df['ks_pz']**2)
        # mass squared
        df['ks_m2'] = df['ks_E']**2 - df['ks_px']**2 - df['ks_py']**2 - df['ks_pz']**2

        # delta++ (pip1 + proton)
        # 4-vector components
        df['delta_E'] = df['pip1_E'] + df['p_E']
        df['delta_px'] = df['pip1_px'] + df['p_px']
        df['delta_py'] = df['pip1_py'] + df['p_py']
        df['delta_pz'] = df['pip1_pz'] + df['p_pz']
        # total mass
        df['delta_m'] = np.sqrt(df['delta_E']**2 - df['delta_px']**2 - df['delta_py']**2 - df['delta_pz']**2)
        # mass squared
        df['delta_m2'] = df['delta_E']**2 - df['delta_px']**2 - df['delta_py']**2 - df['delta_pz']**2

        





        # Create a histogram
        h_ks_m = ROOT.TH1F("h_ks_m", "Invariant Mass of K_S; Mass (GeV/c^2); Counts", 50, min(df['ks_m']), max(df['ks_m']))
        # Create a canvas
        c = ROOT.TCanvas("c", "c", 800, 600)
        # Draw the canvas

        # Fill the histogram
        for value in df['ks_m']:
            h_ks_m.Fill(value)

        # Draw the histogram onto the canvas
        h_ks_m.Draw()
        # Update the canvas
        c.Update()
        # Keep the canvas open
        c.WaitPrimitive()


# Replace the filename and treename with your actual file and tree names
analysis("KKpi_flat_bestX2_2017.root", "pipkmks__B4_M16;1")
